In [1]:
# Initialization

import math
from datetime import datetime
import openpyxl as opxl
from openpyxl.utils.dataframe import dataframe_to_rows
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import model as ty
import proc as pr
import matplotlib.ticker as mtick
import sklearn.metrics

from IPython.core.display import HTML

import heat_transfer as bht

import fluids as fds
import ht 

import general as gen

import os

import scipy.integrate as integrate
import scipy.optimize as sco

import networkx as nx

import plotly.graph_objects as go

import plot_functions_here as pfun

import sys
sys.path.insert(0,r'C:\Users\valen\Dropbox (DualSun)\PC\Documents\GitHub\RD-systems-and-test-benches\utils')


In [24]:
400/3600

0.1111111111111111

In [37]:
v45_man = gen.duct(type="tube",diameter=0.018, length=1.,thickness=0.002,roughness=0.0015*1E-3,fluid='INCOMP::MPG[0.]',mdot=0.220)

In [38]:
v45_man.lv

0.866817121253945

In [35]:
v45_riser = gen.duct(type="rectangle", width=0.002, height=0.0028, length=1.,thickness=0.002,roughness=0.0015*1E-3,fluid='INCOMP::MPG[0.]',mdot=400/(16*4)/3600)

In [36]:
v45_riser.lv

0.31083447520844176

In [4]:
inox_btp = gen.duct(type="tube",diameter=0.020, length=20.,thickness=0.002,roughness=0.0015*1E-3,fluid='INCOMP::MPG[0.4]',mdot=2000/3600)

In [ ]:
inox_btp.compute_flow()

In [ ]:
inox_btp = gen.duct(type="tube",diameter=0.0165, length=20.,thickness=0.002,roughness=0.0015*1E-3,fluid='INCOMP::MPG[0.4]',mdot=800/3600)

In [ ]:
inox_btp.Re

In [ ]:
inox = gen.duct("tube",0.020,0.,0.,20.,0.002,0.0015*1E-3,280/3600,fluid='INCOMP::MPG[0.4]')

In [ ]:
inox.set_thermal(0.5,0.,0.033)

In [ ]:
# inox = gen.duct("tube",0.017,0.,0.,6.,0.001,0.0015*1E-3,180/3600,fluid='INCOMP::MPG[0.4]')

In [ ]:
T_amb = 13+273.15
T_fluid_m = -3+273.15

In [ ]:
inox.compute_heat_transfer(T_fluid_m,T_amb,7)

In [ ]:
inox.compute_heat_transfer_cylinder_free_convection(T_fluid_m,T_amb)

In [ ]:
# create dataframe with one column "T_fluid_m" filled with a list
# of values from 0 to 100
T_amb = 20+273.15
df = pd.DataFrame({"T_fluid_m": np.arange(-20+273.15, 25+273.15)})

df["T_m - T_amb"] = df["T_fluid_m"] - T_amb

for row in df:
    print(row)

In [ ]:
for index, row in df.iterrows():
    row["T_fluid_m"] = 1

In [ ]:
df

In [ ]:
df["Heat transfer"] = df.apply(lambda row: inox.compute_heat_transfer_cylinder_free_convection(row["T_fluid_m"], T_amb), axis=1)

In [ ]:
inox.T_ins-273.15

In [ ]:
inox.h_ext

In [ ]:
inox.h_int

In [ ]:
inox.R_tot

In [ ]:
inox.T_ins-273.15

In [ ]:
T_tube 

In [ ]:
inox.T_ins-273.15

In [ ]:
bht.back_h_cylinder(0+273.15,13+273.15,0.020)

In [ ]:
inox2 = gen.duct("tube",0.020,0.,0.,5.,0.002,0.0015*1E-3,280/3600,fluid='INCOMP::MPG[0.4]')
inox2.set_thermal(0.5,0.01,0.033)
inox2.compute_heat_transfer(-3+273.15,25+273.15,7)

In [ ]:
tube = gen.duct("rectangle",0,0.003,0.003,1.6,0.00035,0.0015*1E-3,120/300/3600,fluid='INCOMP::MPG[0.4]')

In [ ]:
man = gen.duct("tube",0.018,0.,0.,1.,0.004,0.0015*1E-3,120/1.5/3600,fluid='INCOMP::MPG[0.4]')

In [ ]:
tube2 = gen.duct("tube",0.003,0.,0.,1.,0.00035,0.0015*1E-3,120/300/3600,fluid='INCOMP::MPG[0.4]')

In [ ]:
tube2.h_int

In [ ]:
tube.h_int

In [ ]:
man.h_int

In [ ]:
man.Re

In [ ]:
color_lists = pfun.generate_several_color_lists(2,4)
print(color_lists)

In [ ]:
def N(lambd,w,L_tube,D):
    N_real = w/(lambd+D)
    N_virtual = N_real*L_tube/w
    return N_real,N_virtual

In [ ]:
theta = 45.

lambd = 0.0001
w = 1
L_tube = 1.342

delta = 0.003
k = 226
delta_int = 0.003

T_amb = 0.+273.15

In [ ]:
for i,L in enumerate([0.020,0.050]):

    L_fin = L

    for j,T in enumerate([-8,-6,-4,-2]):

        T_abs = T+273.15

        D_list = np.linspace(0.001,0.03,20)
        Q_list = []

        for D in D_list:
            N_real,N_virtual = N(lambd,w,L_tube,D)
            S = w*L_tube
            h_S = bht.back_h_simple(T_abs,T_amb,theta,L_tube)

            ref_Q = -h_S*S*(T_abs-T_amb)

            h = bht.back_h_fins(T_abs,T_amb,theta,L_tube,D,L_fin)
            Bi, gamma_0_int = ty.gamma0int(N_virtual,L_fin,lambd,k,delta,delta_int,L_tube,h)

            S = w*L_tube - N_real*lambd


            Q_list.append(-gamma_0_int*(T_abs-T_amb)*300 -h*S*(T_abs-T_amb) - ref_Q)

        plt.plot(np.array(D_list)*1000,Q_list,label="L_fin = "+str(int(L*1000))+" mm, "+str(T_abs - T_amb)+" K",color=color_lists[i][j])

plt.axvline(x=14, color='grey', linestyle='--') 
plt.axvline(x=18, color='grey', linestyle='--') 
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2)

plt.xlabel("Fin spacing (mm)")
plt.ylabel("Additional heat transfer from fins (W)")
plt.title("V4 panel")
plt.show()